In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/금융시장의빅데이터분석/8. ML/Homework

/content/drive/MyDrive/Colab Notebooks/금융시장의빅데이터분석/8. ML/Homework


## Exercise - Classification Models

- In this exercise, we will try to predict the direction of stock price movements using technical indicators.
- Technical indicator?
    - Technical indicators are heuristic or pattern-based signals produced by the price, volume, and/or open interest of a security or contract used by traders who follow technical analysis.
    - By analyzing historical data, technical analysts use indicators to predict future price movements. Examples of common technical indicators include the Relative Strength Index (RSI), Money Flow Index (MFI), stochastics, moving average convergence divergence (MACD), and Bollinger Bands®.
    - For more detail (https://www.investopedia.com/terms/t/technicalindicator.asp).
    
- Data: AAPL stock (about 20y of daily data)
    - y: binary
        - 1 if price moves up
        - 0 otherwise
    - Open, High, Low, Close: Price data
    - the rest: technical indicators (about 56 indicators)
    
- Based on the procedure we have learned in class (cross-validation, hyperparameter tuning, etc), train 3 models and compare their results.

In [3]:
import pandas as pd
import numpy as np

df=pd.read_csv('exercise.csv', index_col=0, parse_dates=True)
df.head()

,Open,High,Low,Close,volume,volume_cmf,volume_fi,volume_em,volume_sma_em,volume_vpt,...,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,y
Date,,,,,,,,,,,,,,,,,,,,,
2001-04-16,0.394464,0.400000,0.372500,0.382857,2.852248e+08,0.033422,5.742850e+05,-0.000078,0.000009,-3.966024e+06,...,63.878281,-42.765900,0.016932,-3.292812,-9.519884,-3.168121,-6.351763,-4.371102,-4.469513,0.0
2001-04-17,0.378571,0.378750,0.350000,0.364286,6.851992e+08,-0.000052,-1.325588e+06,-0.000092,0.000002,-4.570397e+07,...,55.247096,-64.892923,0.015173,-9.453895,-3.299504,-3.194397,-0.105106,-4.850633,-4.972225,1.0
2001-04-18,0.385179,0.430000,0.376429,0.406964,1.100842e+09,0.062106,5.575460e+06,0.000189,0.000018,9.573291e+07,...,56.046121,-24.202816,0.016178,3.262309,7.801987,-0.995121,8.797107,11.715520,11.078545,1.0
2001-04-19,0.456250,0.459821,0.421429,0.459286,1.873670e+09,0.205960,1.878385e+07,0.000077,0.000020,3.698609e+08,...,70.158759,-0.427985,0.021710,19.129218,23.668194,3.937542,19.730652,12.856665,12.094838,0.0
2001-04-20,0.445179,0.457679,0.439286,0.447143,6.934032e+08,0.147344,1.489758e+07,0.000021,0.000025,2.225587e+08,...,88.408934,-10.142398,0.029500,23.715303,20.996967,7.349427,13.647540,-2.643890,-2.679469,0.0


In [4]:
# unit root test

from statsmodels.tsa.stattools import adfuller

for col in df.columns:
    if adfuller(df[col])[1] > 0.05: print(col)
    
df = df.drop(['Open', 'High', 'Low', 'Close'], axis=1)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Open
High
Low
Close


In [5]:
# standardize

# (1) fit_transform the whole data 

# (2) fit on the train data 

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
y = df['y']
X = df.drop('y', axis=1)
temp = scaler.fit_transform(X)
X = pd.DataFrame(temp, index=X.index, columns = X.columns)

In [6]:
from sklearn.model_selection import train_test_split, KFold

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
import warnings 

warnings.filterwarnings(action='ignore')

In [8]:
# Logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV



penalties = ['l1', 'l2', 'elasticnet', 'none'] #elatsicnet : l1, l2 다씀 / None은 안씀
intercepts = [True, False] #상수항 있는지 판단
C = [0.001, 0.01, 0.1, 0., 1.0, 10.0, 100.0, 1000.0] #penalty term
model = LogisticRegression()

hyperparameters = {'fit_intercept':intercepts,
                   'penalty':penalties,
                   'C':C}

gsearch = GridSearchCV(model, hyperparameters, verbose=1)
gsearch.fit(X_train, y_train, )

Fitting 5 folds for each of 64 candidates, totalling 320 fits


GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 0.0, 1.0, 10.0, 100.0, 1000.0],
                         'fit_intercept': [True, False],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none']},
             verbose=1)

In [10]:
gsearch.best_params_

{'C': 0.01, 'fit_intercept': True, 'penalty': 'l2'}

In [12]:
from sklearn.metrics import classification_report

model = LogisticRegression(**gsearch.best_params_)
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(classification_report(y_test,predicted))

              precision    recall  f1-score   support

         0.0       0.49      0.34      0.40       708
         1.0       0.53      0.69      0.60       780

    accuracy                           0.52      1488
   macro avg       0.51      0.51      0.50      1488
weighted avg       0.52      0.52      0.51      1488



In [ ]:
# SVM
from sklearn.svm import SVC

C = [0.01, 0.1, 0., 1.0, 10.0]
degrees = [2,3,5]
coefs = [0,1,10]
#kernels = ['linear', 'poly', 'rbf', 'sigmoid']
kernels = ['rbf', 'sigmoid']
gammas = ['scale', 'auto']
model = SVC()

hyperparameters = { 'kernel': kernels,
                   'coef0': coefs,
                   'degree':degrees,
                   'gamma': gammas,
                   'C':C}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

gsearch = GridSearchCV(model, hyperparameters, verbose=5)
gsearch.fit(X_train, y_train, )

Fitting 5 folds for each of 180 candidates, totalling 900 fits
[CV 1/5] END C=0.01, coef0=0, degree=2, gamma=scale, kernel=rbf;, score=0.524 total time=   0.9s
[CV 2/5] END C=0.01, coef0=0, degree=2, gamma=scale, kernel=rbf;, score=0.524 total time=   0.9s
[CV 3/5] END C=0.01, coef0=0, degree=2, gamma=scale, kernel=rbf;, score=0.524 total time=   0.9s
[CV 4/5] END C=0.01, coef0=0, degree=2, gamma=scale, kernel=rbf;, score=0.524 total time=   0.8s
[CV 5/5] END C=0.01, coef0=0, degree=2, gamma=scale, kernel=rbf;, score=0.524 total time=   0.9s
[CV 1/5] END C=0.01, coef0=0, degree=2, gamma=scale, kernel=sigmoid;, score=0.524 total time=   0.9s
[CV 2/5] END C=0.01, coef0=0, degree=2, gamma=scale, kernel=sigmoid;, score=0.524 total time=   0.9s
[CV 3/5] END C=0.01, coef0=0, degree=2, gamma=scale, kernel=sigmoid;, score=0.524 total time=   0.9s
[CV 4/5] END C=0.01, coef0=0, degree=2, gamma=scale, kernel=sigmoid;, score=0.524 total time=   0.9s
[CV 5/5] END C=0.01, coef0=0, degree=2, gamma=sc

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.01, 0.1, 0.0, 1.0, 10.0], 'coef0': [0, 1, 10],
                         'degree': [2, 3, 5], 'gamma': ['scale', 'auto'],
                         'kernel': ['rbf', 'sigmoid']},
             verbose=5)

In [ ]:
mod = gsearch.best_estimator_.fit(X_train, y_train)
print(classification_report(y_test, mod.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.50      0.35      0.41       708
         1.0       0.54      0.68      0.60       780

    accuracy                           0.52      1488
   macro avg       0.52      0.51      0.50      1488
weighted avg       0.52      0.52      0.51      1488



- Accuracy : 정답률이 얼마나 되는가? $\rightarrow$ 52%의 정답률
- 1에 대해 recall, f1-score가 높음


**지표 해석**

- True class: Actual values of $y_i$
- Predicted class: Predicted values of $y_i$ ($\hat{y}_i$)
- Accuracy: ${ TP + TN \over TP + TN + FP +FN }$ (모형 전반의 성능)
    - Measures the overall performance of the model
    - Sensitive to class imbalance $\Rightarrow$ We need alternative measures
    - Class Imbalance : Positive와 Negative의 비율이 비슷하지 않으면 문제?가 생김 

- Recall: ${TP \over TP + FN}$ (True Positive + False Negative 중 TP 비율)
- Precision: ${TP \over TP  + FP}$ (TP + FP 중 TP 비율, Positive라고 예측할때 옳게 Positive라고 예측했는가?)
- F1-score: $2 \cdot {recall \cdot precision \over recall + precision} $ (Recall과 Precision의 조화평균)

- Positive : 돈을 벌수 있는 타이밍 -> 옳게 베팅한 비율을 높이는 것이 투자에서 수익률을 높이는 방법

In [ ]:
# Random forest

from sklearn.ensemble import RandomForestClassifier

nestimators = [10,20,50,100]
criterion = ['gini', 'entropy']
maxdepth = [3,5,10,20]
minsampleleafs = [0.005 ,0.01, 0.05, 0.10]
maxfeatures = [2,3,5,11]

model = RandomForestClassifier(random_state=42)

hyperparameters = {'n_estimators': nestimators,
                   'max_features': maxfeatures,
                   'criterion': criterion,
                   'max_depth': maxdepth,
                   'min_samples_leaf':minsampleleafs
                  }

gsearch = GridSearchCV(model, hyperparameters, verbose=1)
gsearch.fit(X_train, y_train, )


Fitting 5 folds for each of 512 candidates, totalling 2560 fits


GridSearchCV(estimator=RandomForestClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 10, 20],
                         'max_features': [2, 3, 5, 11],
                         'min_samples_leaf': [0.005, 0.01, 0.05, 0.1],
                         'n_estimators': [10, 20, 50, 100]},
             verbose=1)

In [ ]:
mod = gsearch.best_estimator_.fit(X_train, y_train)
print(classification_report(y_test, mod.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.49      0.27      0.35       708
         1.0       0.53      0.75      0.62       780

    accuracy                           0.52      1488
   macro avg       0.51      0.51      0.48      1488
weighted avg       0.51      0.52      0.49      1488



문제 형태 :
- 분류 모형(데이터)를 갖고 옴
- 이에 대해 분류, 모형별로 정리
- 수업시간에 설명한 코드를 긁어와서 붙일 수 있는 능력